In [1]:
import csv 
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import numpy as np
import nltk
import itertools
from nltk.corpus import stopwords
import re
import NFile_utils

nltk.download("stopwords")
nltk.download('punkt')

Obtained my token successfully!
{'data': 2, 'error': 'task_on_executing', 'error_Description': "Task is running! You don't need to start again."}
{'data': 2, 'error': 'task_on_executing', 'error_Description': "Task is running! You don't need to start again."}
{'data': 1, 'error': 'success', 'error_Description': 'Task is started successfully!'}


[nltk_data] Downloading package stopwords to /home/kasem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/kasem/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
Nfiles_df_list, first_extract, second_extract = NFile_utils.Nfiles_df_list, NFile_utils.first_extract_df, NFile_utils.second_extract_df

In [124]:
xls = pd.ExcelFile('strings stems.xlsx')
first = pd.read_excel(xls, 'first').dropna(axis=1, how='all').dropna(axis=0, how='all')
parent_second = pd.read_excel(xls, 'parent second').dropna(axis=1, how='all').dropna(axis=0, how='all')
bad_keywords= pd.read_excel(xls, 'bad keywords').dropna(axis=1, how='all').dropna(axis=0, how='all')
at_least_another= pd.read_excel(xls, 'at least another').dropna(axis=1, how='all').dropna(axis=0, how='all')
tutoring = pd.read_excel(xls, 'Tutoring').dropna(axis=1, how='all').dropna(axis=0, how='all')
payments = pd.read_excel(xls, 'payments').dropna(axis=1, how='all').dropna(axis=0, how='all')
contract = pd.read_excel(xls, 'contract').dropna(axis=1, how='all').dropna(axis=0, how='all')

second_extract =  pd.read_csv('secondextract.csv').dropna(axis=1, how='all').dropna(axis=0, how='all')

pre_process_df_keywords(first)
pre_process_df_keywords(parent_second)
pre_process_df_keywords(bad_keywords)

print('x' in parent_second["sound effects"].values)
contract

False


,name
0,contract
1,NDA


In [128]:
raw_message = """My name is Jack, the designer of Bellorita handbags (bellorita.com). We are working on a 90-second video ad on facebook. We need a video editor to help us edit the video with text, image, and footage overlay to go with the script and spokewoman.

Right now the script (225 words) is ready, but the spokewoman video is not yet, and we need your suggestion on how we need her to performe to smooth your video editing.

If you are interested, please propose with your price.

Please note that this is an easy project, and we want the lowest rate possible. Thank you.

I will pay through cash app and I need to sign an NDA

Sincerely
Jack
""".lower()
message = pre_process(raw_message)

print("Breaking_2") if step_1(raw_message, get_two_word(message), bad_keywords) else False
print(step_1(raw_message, get_two_word(message), bad_keywords))


find_workers(raw_message, message, parent_second), find_workers(raw_message, message, first)

[]


((array(['photoshop', 'video editing'], dtype=object), 'video editing'),
 (array(['a graphic designer', 'someone', 'a video editor'], dtype=object),
  'a video editor'))

In [101]:
fuzz.partial_ratio("solid works", "solidworks")

90

In [39]:
'class a ' in 'class aside'

False

In [40]:
len("""
a v """.lower().split())
from ner import NER

In [41]:
len("""
photoshop needed
""".lower())
extractor = NER()

In [42]:
from ner import NER
text = """compensation: $60/hour
"""
def append_sapces(text):
    if text.find("-"): 
        (text[:text.find("-")] + " " + text[text.find("-")] + " " + text[text.find("-")+1:])
    if text.find("/"):
        (text[:text.find("/")] + " " + text[text.find("/")] + " " + text[text.find("/")+1:])
    return text

In [94]:
extractor.extract_entities_dict(append_sapces(text))

{'money': '60'}

In [95]:
extractor.extract_entities_dict(append_sapces((text)))

{'money': '60'}

In [129]:
find_matching_key_word(message, contract)

['NDA']